In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
reviews_sample= pd.read_csv(r'C:\Users\shanm\OneDrive\Desktop\Git_data\Sephora_sample\X_reviews_sample.csv',low_memory =True)
reviews_sample.head()

,Unnamed: 0,review_text,label,rating,text_word_count,clean_text,tokenized_text,tokenized_text_2
0,0,Gel type moisturizers aren’t typically my go-t...,1.0,5,51,gel type moisturizers aren’t typically my goto...,"['gel', 'type', 'moistur', '’', 'typic', 'goto...",gel type moistur ’ typic goto one impress gel ...
1,1,smells great! works really well on my acne sca...,1.0,4,44,smells great works really well on my acne scar...,"['smell', 'great', 'work', 'realli', 'well', '...",smell great work realli well acn scare need co...
2,2,Caused me so many breakouts. I thought this wo...,0.0,1,29,caused me so many breakouts i thought this wou...,"['caus', 'mani', 'breakout', 'thought', 'would...",caus mani breakout thought would good use clea...
3,3,I could not be happier with this mask! I alway...,1.0,5,45,i could not be happier with this mask i always...,"['could', 'happier', 'mask', 'alway', 'use', '...",could happier mask alway use overnight mask am...
4,4,Fastest acting spot reducer I have ever tried....,1.0,5,50,fastest acting spot reducer i have ever tried ...,"['fastest', 'act', 'spot', 'reduc', 'ever', 't...",fastest act spot reduc ever tri got watch one ...


In [3]:
reviews_sample.loc[reviews_sample['text_word_count']==409]

,Unnamed: 0,review_text,label,rating,text_word_count,clean_text,tokenized_text,tokenized_text_2
21278,21278,Shiseido has a great reputation with putting o...,1.0,5,409,shiseido has a great reputation with putting o...,"['shiseido', 'great', 'reput', 'put', 'qualiti...",shiseido great reput put qualiti product glad ...


In [4]:
reviews_sample['text_word_count'].describe()

count    156820.000000
mean         58.845606
std          41.760496
min           1.000000
25%          31.000000
50%          49.000000
75%          75.000000
max         409.000000
Name: text_word_count, dtype: float64

In [5]:
X= reviews_sample['clean_text']
y = reviews_sample['label'] 

In [6]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, TextVectorization
from tensorflow.keras.optimizers import Adam

In [8]:
max_features = 2500  # Maximum vocab size.
max_len = 200  # Sequence length to pad the outputs to.

# Create the layer.
vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

# Now that the vocab layer has been created, call `adapt` on the
# text-only dataset to create the vocabulary. You don't have to batch,
# but for large datasets this means we're not keeping spare copies of
# the dataset.
vectorize_layer.adapt(X_train)

In [9]:
x_train = vectorize_layer(X_train)

In [10]:
x_test = vectorize_layer(X_test)

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # encoding
y_train_label= le.fit_transform(y_train)
y_test_label= le.transform(y_test)

In [13]:
x_train.shape,y_train_label.shape

(TensorShape([31364, 200]), (31364,))

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Embedding(max_features,64,input_length = 200))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 64)           160000    
                                                                 
 bidirectional (Bidirectiona  (None, 200, 128)         66048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 25)                1625      
                                                                 
 dense_4 (Dense)             (None, 1)                 26        
                                                      

In [16]:
model.fit(x_train,y_train_label, epochs = 3)

Epoch 1/3
981/981 [==============================] - 135s 134ms/step - loss: 0.6934 - accuracy: 0.5004
Epoch 2/3
981/981 [==============================] - 133s 136ms/step - loss: 0.6933 - accuracy: 0.4989
Epoch 3/3
981/981 [==============================] - 130s 133ms/step - loss: 0.6931 - accuracy: 0.5023


In [17]:
model.predict(vectorize_layer('Gel type moisturizers aren’t typically my'))

ValueError: in user code:

    File "C:\Users\shanm\anaconda3\envs\keras_env\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\shanm\anaconda3\envs\keras_env\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\shanm\anaconda3\envs\keras_env\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\shanm\anaconda3\envs\keras_env\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\shanm\anaconda3\envs\keras_env\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\shanm\anaconda3\envs\keras_env\lib\site-packages\keras\engine\input_spec.p